[View in Colaboratory](https://colab.research.google.com/github/vineetjoshi253/Image-based-Indian-Monument-Recognition-using-Convoluted-Neural-Networks/blob/master/CNN.ipynb)

In [0]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
train_downloaded = drive.CreateFile({'id': '1g8ekKhFpiih5IpFNeIiUlpKIvCD7KYs_'})
train_downloaded.GetContentFile('X.npy')
test_downloaded = drive.CreateFile({'id': '1ifWS_MNnl4YN5Ae5K7MXBEJDNiwK84HZ'})
test_downloaded.GetContentFile('Y.npy')  

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [28]:
import numpy as np
X=np.load('X.npy')
Y=np.load('Y.npy')
Z=np.zeros(712)
index=0
for i in range(0,712):
  if(Y[i]==b'charminar'):
    Z[index]=0
    index+=1
  elif(Y[i]==b'qutub-minar'):
    Z[index]=1
    index+=1
  elif(Y[i]==b'india-gate'):
    Z[index]=2
    index+=1
  elif(Y[i]==b'taj-mahal'):
    Z[index]=3
    index+=1
  else:
    Z[index]=4
    index+=1
print(X.shape)

(712, 9216)


In [0]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator

x_train,x_test,y_train,y_test=train_test_split(X,Z,random_state=42,test_size=0.30)
img_rows, img_cols, img_colors = 96, 96, 1

num_classes=11
y_train_one_hot = keras.utils.to_categorical(y_train)
y_test_one_hot = keras.utils.to_categorical(y_test)


x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, img_colors)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, img_colors)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(96,96,1)))
model.add(BatchNormalization(axis=-1, momentum=0.001))
model.add(Activation('sigmoid'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(axis=-1, momentum=0.001))
model.add(Activation('sigmoid'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3, 3)))
model.add(BatchNormalization(axis=-1, momentum=0.001))
model.add(Activation('sigmoid'))
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization(axis=-1, momentum=0.001))
model.add(Activation('sigmoid'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

# Fully connected layer
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(5))

model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

model.fit(x_train,y_train_one_hot,validation_data=(x_test,y_test_one_hot),verbose=1,batch_size=20,epochs=100)


In [30]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 98, 64)        640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 98, 98, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 47, 32)        18464     
_________________________________________________________________
batch_normalization_2 (Batch (None, 47, 47, 32)        128       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 16)        4624      
__________

In [0]:
model.fit(x_train,y_train_one_hot,validation_data=(x_test,y_test_one_hot),verbose=1,batch_size=32,epochs=100)